In [1]:
%pip install milvus -q
from milvus import default_server

# Stop existing server
default_server.stop()

# Start server again
default_server.start()

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("Milvus-Spark-Integration") \
	.config("spark.jars", "/Users/emircan/spark-milvus-1.0.0-SNAPSHOT.jar") \
	.config("spark.driver.memory", "4g") \
    .getOrCreate()

Note: you may need to restart the kernel to use updated packages.


25/01/08 20:09:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
spark.stop()

In [2]:
# Connect to Milvus
import milvus
client = milvus.MilvusServer(host="localhost", port=default_server.listen_port)

In [8]:
from pymilvus import connections, utility

# Connect to Milvus
connections.connect(host='localhost', port='19530')

# List all collections
collections = utility.list_collections()
print("Available collections:", collections)

Available collections: ['embeddings_collection', 'emb']


In [43]:
from pymilvus import Collection, connections
# connections.connect(host='localhost', port='19530')

collection = Collection("emb")      # Use your collection name
collection.load()

In [50]:
import ollama

# Initialize Ollama client
ollama_client = ollama.Client()

# First pull the model (run this once)
# ollama pull granite-embedding:278m # For multilingual support, 278M parameters

# Sample text to generate embeddings for
text = "Dil: Türkçe \n çek iptali"

# Generate embeddings
response = ollama_client.embeddings(
    model="snowflake-arctic-embed2",
    prompt=text
)

# Get the embedding vector
embedding = response["embedding"]
print(embedding)

[-0.01012931764125824, -0.42942678928375244, 1.1799230575561523, -0.12760469317436218, -0.19621479511260986, 0.3648108243942261, 0.67621910572052, 0.5245853066444397, -0.28668659925460815, -0.5714994072914124, -0.40392377972602844, -0.0998082384467125, 0.195916548371315, 0.8253580927848816, -1.0778473615646362, -0.6298412680625916, -0.14134734869003296, 0.27160409092903137, -0.16578228771686554, 0.49073028564453125, -0.1807873398065567, 1.0718376636505127, 0.6703019738197327, -0.370166152715683, 0.21638521552085876, 0.4277220070362091, 0.11306656152009964, -0.3522321879863739, -0.40512165427207947, -0.49301883578300476, 0.32206273078918457, 0.14624875783920288, 0.4275433421134949, -0.8901683688163757, -0.5156583786010742, 0.06519689410924911, -0.3732062578201294, -0.13281559944152832, -1.3231583833694458, 1.3201791048049927, -0.004548035562038422, 0.7964673638343811, 0.6749289035797119, 0.12077898532152176, 0.21910996735095978, -0.5288488268852234, -0.1119697093963623, -0.5592564940452

In [51]:
import numpy as np
query_vector = np.array(embedding)
query_vector = np.array(query_vector) / np.linalg.norm(query_vector)

search_params = {
    "metric_type": "IP",           # IP for normalized vectors (cosine similarity)
    "params": {"nprobe": 10}       # Number of clusters to search
}


res = collection.search(
	data=[query_vector.tolist()],           # List of vectors to search for
	anns_field="vec",              # Vector field name
	param=search_params,
	limit=5,                   # Number of results to return
	output_fields=["id", "text"]   # Additional fields to return
)


In [52]:
print(res[0][0].entity.text)



T.C.
İSTANBUL
4. ASLİYE TİCARET MAHKEMESİ

ESAS NO : 2023/173 Esas
KARAR NO : 2023/236

DAVA : Kıymetli Evrak İptali (Çek İptali (Hasımsız))
DAVA TARİHİ : 13/03/2023
KARAR TARİHİ : 16/03/2023

Mahkememizde görülmekte olan Kıymetli Evrak İptali (Çek İptali (Hasımsız)) davasının yapılan açık yargılaması sonunda,
GEREĞİ DÜŞÜNÜLDÜ:
Davacı şirket yetkilisi dava dilekçesinde özetle, ... A.Ş'den alınmış olan çek defterinin keşide edilmemiş son yaprağı ile birlikte kayıp edildiğini ileri sürerek çekin iptaline karar verilmesini talep etmiş ise de, dava dilekçesinde de belirtildiği gibi çekin keşide edilmediği, dolayısıyla kıymetli evrak niteliğini kazanmadığı gibi 6102 sayılı TTK'nın 651/2.maddesi gereğince kıymetli evrak zayi olduğu anda ancak çek üzerinde hak sahibi olan kişinin iptal davası açma hakkının bulunduğu keşidecinin çek iptal davası açma hakkının bulunmadığı (Yargıtay 11.Hukuk Dairesinin 30/10/2013 tarih 2013/3036 esas 2013/19155 karar sayılı ilamının da aynı yönde olduğu) dikka

25/01/08 21:42:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 465187 ms exceeds timeout 120000 ms
25/01/08 21:42:14 WARN SparkContext: Killing executors is not supported by current scheduler.
25/01/08 21:42:22 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [46]:
print(res[0][1].entity.text)



T.C.
İSTANBUL
3. ASLİYE TİCARET MAHKEMESİ

ESAS NO : 2023/743 Esas
KARAR NO : 2024/215

DAVA : İflas (Adi Takipten Doğan İtirazın Kaldırılması Ve İflas (İİK 156))
DAVA TARİHİ : 16/11/2023
KARAR TARİHİ : 21/03/2024

Mahkememizde açılan davanın yapılan yargılaması sonunda;
GEREĞİ DÜŞÜNÜLDÜ:
Davacı vekili dava dilekçesinde ve aşamalardaki beyanlarında özetle; müvekkili şirketin davalı şirketten olan fatura alacağı nedeni ile davalı-borçlu aleyhine önce ... 12. İcra Müdürlüğü'nün ...esas sayılı dosyası ile ilamsız icra takibi başlatıldığını, davalının itirazı üzerine takibin durduğunu, anılan dosyanın adi iflas yolu ile takip olarak değiştirildiğini, takip şeklinin değiştirilmiş olması nedeni ile davalı borçlu vekiline iflas yolu ile adi takipte ödeme emri gönderildiğini, borçlu tarafından işbu ödeme emrine de itiraz edildiğini, bilahare davalı-borçlu aleyhine ... 11. İcra Müdürlüğü'nün... esas sayılı takip dosyası ile iflas takibi başlatıldığını borçluya ödeme emrinin 29.11.2022 tarihin

In [53]:
collection.delete("where id != ''")

25/01/08 21:55:23 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

MilvusException: <MilvusException: (code=65535, message=empty grpc client: failed to connect 161.9.97.105:53100, reason: context deadline exceeded)>

25/01/08 21:56:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j